<a href="https://colab.research.google.com/github/m-mehdi/Python101/blob/master/ApacheSpark_03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Manipulating Data using Apache Spark

In this notebook, we are going to get our hands dirty with Spark DataFrame API to perform common data operations.

In [1]:
!pip install pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

     |████████████████████████████████| 204.2MB 69kB/s 
     |████████████████████████████████| 204kB 29.4MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.1-py2.py3-none-any.whl size=204612243 sha256=e38c163bb2a8a55b674a2ef2220c079aa3f1db55937775c4d12bc38d84229f63
  Stored in directory: /root/.cache/pip/wheels/5e/bd/07/031766ca628adec8435bb40f0bd83bb676ce65ff4007f8e73f
Successfully built pyspark


In [2]:
spark

If you take a look at dataset folder, you will see `flights.csv` that contains a row for every flight that left Portland International Airport (PDX) or Seattle-Tacoma International Airport (SEA) in 2014 and 2015.

In the first step, we should create a DataFrame using `flights.csv` file and then create a table (temporaray view) for querying flights by using SQL commands. Let's do it.

In [5]:
import os
MAIN_DIRECTORY = os.getcwd()
file_path =MAIN_DIRECTORY+"/dataset/flights.csv"

In [6]:
df_flights = spark.read.format("csv").option("header","true").option('inferSchema','true').load(file_path)

In [7]:
df_flights.createOrReplaceTempView('flights')

### Exercise 1: Use SQL to get the first five rows of the flights table and save the result to flights5, finally show the results. 

In [8]:
query = "SELECT * FROM flights LIMIT 5"
flights5 = spark.sql(query)
flights5.show()

+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
|2014|   12|  8|     658|       -7|     935|       -5|     VX| N846VA|  1780|   SEA| LAX|     132|     954|   6|    58|
|2014|    1| 22|    1040|        5|    1505|        5|     AS| N559AS|   851|   SEA| HNL|     360|    2677|  10|    40|
|2014|    3|  9|    1443|       -2|    1652|        2|     VX| N847VA|   755|   SEA| SFO|     111|     679|  14|    43|
|2014|    4|  9|    1705|       45|    1839|       34|     WN| N360SW|   344|   PDX| SJC|      83|     569|  17|     5|
|2014|    3|  9|     754|       -1|    1015|        1|     AS| N612AS|   522|   SEA| BUR|     127|     937|   7|    54|
+----+-----+---+--------+---------+-----

### Exercise 2: Write a query that counts the number of flights to each airport from SEA and PDX.

In [9]:
query = "SELECT origin, dest, count(*) as NUM_FLIGHTS FROM flights GROUP BY origin, dest"
flight_counts = spark.sql(query)
flight_counts.show()

+------+----+-----------+
|origin|dest|NUM_FLIGHTS|
+------+----+-----------+
|   SEA| RNO|          8|
|   SEA| DTW|         98|
|   SEA| CLE|          2|
|   SEA| LAX|        450|
|   PDX| SEA|        144|
|   SEA| BLI|          5|
|   PDX| IAH|         57|
|   PDX| PHX|        209|
|   SEA| SLC|        225|
|   SEA| SBA|         23|
|   SEA| BWI|         29|
|   PDX| IAD|         23|
|   PDX| SFO|        305|
|   SEA| KOA|         40|
|   PDX| MCI|         15|
|   SEA| SJC|        213|
|   SEA| ABQ|         43|
|   SEA| SAT|         18|
|   PDX| ONT|         57|
|   SEA| LAS|        364|
+------+----+-----------+
only showing top 20 rows



In [10]:
df_flights.groupBy(['origin','dest']).count().show()

+------+----+-----+
|origin|dest|count|
+------+----+-----+
|   SEA| RNO|    8|
|   SEA| DTW|   98|
|   SEA| CLE|    2|
|   SEA| LAX|  450|
|   PDX| SEA|  144|
|   SEA| BLI|    5|
|   PDX| IAH|   57|
|   PDX| PHX|  209|
|   SEA| SLC|  225|
|   SEA| SBA|   23|
|   SEA| BWI|   29|
|   PDX| IAD|   23|
|   PDX| SFO|  305|
|   SEA| KOA|   40|
|   PDX| MCI|   15|
|   SEA| SJC|  213|
|   SEA| ABQ|   43|
|   SEA| SAT|   18|
|   PDX| ONT|   57|
|   SEA| LAS|  364|
+------+----+-----+
only showing top 20 rows



### Exercise 3: Write a piece of code to create a DataFrame using `airports.csv`, this file contains information about different airports all over the world. 

In [11]:
file_path = MAIN_DIRECTORY+"/dataset/airports.csv"
df_airports = spark.read.csv(file_path, header=True, inferSchema=True)
df_airports.show(10)

+---+--------------------+----------+------------+----+---+---+
|faa|                name|       lat|         lon| alt| tz|dst|
+---+--------------------+----------+------------+----+---+---+
|04G|   Lansdowne Airport|41.1304722| -80.6195833|1044| -5|  A|
|06A|Moton Field Munic...|32.4605722| -85.6800278| 264| -5|  A|
|06C| Schaumburg Regional|41.9893408| -88.1012428| 801| -6|  A|
|06N|     Randall Airport| 41.431912| -74.3915611| 523| -5|  A|
|09J|Jekyll Island Air...|31.0744722| -81.4277778|  11| -4|  A|
|0A9|Elizabethton Muni...|36.3712222| -82.1734167|1593| -4|  A|
|0G6|Williams County A...|41.4673056| -84.5067778| 730| -5|  A|
|0G7|Finger Lakes Regi...|42.8835647| -76.7812318| 492| -5|  A|
|0P2|Shoestring Aviati...|39.7948244| -76.6471914|1000| -5|  U|
|0S9|Jefferson County ...|48.0538086|-122.8106436| 108| -8|  A|
+---+--------------------+----------+------------+----+---+---+
only showing top 10 rows



Let's look at performing column-wise operations. In Apache Spark, you can do this using the `.withColumn(colName, col)`  which returns a new DataFrame by adding a column or replacing the existing column that has the same name.

*Parameters*:  
- **colName** – string, name of the new column.
- **col** – a Column expression for the new column. 

The new `column` must be an object of class Column. Creating one of these is as easy as extracting a column from your DataFrame using `df.colName`.
Apache Spark DataFrame is **immutable**. Immutable means that it can't be changed, and so columns can't be updated in place.
For example:
```python
df = df.withColumn("newCol", df.oldCol + 1)
```
The above code creates a DataFrame with the same columns as df plus a new column, `newCol`, where every entry is equal to the corresponding entry from `oldCol`, plus one.

Sometimes we have to change a column data type to another one, in this case, we can use the following code:
```python
from pyspark.sql.functions import col
df_name = df_name.withColumn("columnName", col("columnName").cast("DataType"))
```

### Exercise 4: Update `flights` DataFrame to include a new column called `duration_hrs`, that contains the duration of each flight in hours.

In [13]:
df_flights.show(3)

+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
|2014|   12|  8|     658|       -7|     935|       -5|     VX| N846VA|  1780|   SEA| LAX|     132|     954|   6|    58|
|2014|    1| 22|    1040|        5|    1505|        5|     AS| N559AS|   851|   SEA| HNL|     360|    2677|  10|    40|
|2014|    3|  9|    1443|       -2|    1652|        2|     VX| N847VA|   755|   SEA| SFO|     111|     679|  14|    43|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
only showing top 3 rows



In [18]:
from pyspark.sql.functions import col
df_flights = df_flights.withColumn('air_time',col('air_time').cast('integer'))

In [19]:
df_flights = df_flights.withColumn('duration_hrs', df_flights.air_time/60)

In [15]:
df_flights.columns

['year',
 'month',
 'day',
 'dep_time',
 'dep_delay',
 'arr_time',
 'arr_delay',
 'carrier',
 'tailnum',
 'flight',
 'origin',
 'dest',
 'air_time',
 'distance',
 'hour',
 'minute',
 'duration_hrs']

In [20]:
df_flights.show(3)

+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+------------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|duration_hrs|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+------------+
|2014|   12|  8|     658|       -7|     935|       -5|     VX| N846VA|  1780|   SEA| LAX|     132|     954|   6|    58|         2.2|
|2014|    1| 22|    1040|        5|    1505|        5|     AS| N559AS|   851|   SEA| HNL|     360|    2677|  10|    40|         6.0|
|2014|    3|  9|    1443|       -2|    1652|        2|     VX| N847VA|   755|   SEA| SFO|     111|     679|  14|    43|        1.85|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+------------+
only showing top 3 rows



In [21]:
df_flights.printSchema()

root
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- day: integer (nullable = true)
 |-- dep_time: string (nullable = true)
 |-- dep_delay: string (nullable = true)
 |-- arr_time: string (nullable = true)
 |-- arr_delay: string (nullable = true)
 |-- carrier: string (nullable = true)
 |-- tailnum: string (nullable = true)
 |-- flight: integer (nullable = true)
 |-- origin: string (nullable = true)
 |-- dest: string (nullable = true)
 |-- air_time: integer (nullable = true)
 |-- distance: integer (nullable = true)
 |-- hour: string (nullable = true)
 |-- minute: string (nullable = true)
 |-- duration_hrs: double (nullable = true)



In [23]:
from pyspark.sql.functions import round

In [25]:
df_flights.select(['origin','dest','flight','air_time',round(df_flights.duration_hrs,2).alias('duration_hrs')]).show()

+------+----+------+--------+------------+
|origin|dest|flight|air_time|duration_hrs|
+------+----+------+--------+------------+
|   SEA| LAX|  1780|     132|         2.2|
|   SEA| HNL|   851|     360|         6.0|
|   SEA| SFO|   755|     111|        1.85|
|   PDX| SJC|   344|      83|        1.38|
|   SEA| BUR|   522|     127|        2.12|
|   PDX| DEN|    48|     121|        2.02|
|   PDX| OAK|  1520|      90|         1.5|
|   SEA| SFO|   755|      98|        1.63|
|   SEA| SAN|   490|     135|        2.25|
|   SEA| ORD|    26|     198|         3.3|
|   SEA| LAX|   448|     130|        2.17|
|   SEA| PHX|   656|     154|        2.57|
|   SEA| LAS|   608|     127|        2.12|
|   SEA| ANC|   121|     183|        3.05|
|   SEA| SFO|   306|     129|        2.15|
|   PDX| SFO|  1458|      90|         1.5|
|   SEA| SMF|   368|      76|        1.27|
|   SEA| MDW|   827|     216|         3.6|
|   SEA| BOS|    24|     290|        4.83|
|   PDX| BUR|  3488|     111|        1.85|
+------+---

### Exercise 5: Write a query using the `.filter()` method to find all the flights that flew over 1000 miles.

In [26]:
long_flights = df_flights.filter(df_flights.distance>1000)

In [27]:
long_flights.show(10)

+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+-----------------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|     duration_hrs|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+-----------------+
|2014|    1| 22|    1040|        5|    1505|        5|     AS| N559AS|   851|   SEA| HNL|     360|    2677|  10|    40|              6.0|
|2014|    4| 19|    1236|       -4|    1508|       -7|     AS| N309AS|   490|   SEA| SAN|     135|    1050|  12|    36|             2.25|
|2014|   11| 19|    1812|       -3|    2352|       -4|     AS| N564AS|    26|   SEA| ORD|     198|    1721|  18|    12|              3.3|
|2014|    8|  3|    1120|        0|    1415|        2|     AS| N305AS|   656|   SEA| PHX|     154|    1107|  11|    20|2.566666666666667|
|2014|   11| 12|    2346|       -4

### Exercise 6: Write a query using `.filter()` method, to only keep flights from SEA to PDX. This query should only return `tailnum`, `origin`, and `dest` columns.

In [28]:
df_flights.filter(df_flights.origin =='SEA').filter(df_flights.dest =='PDX')\
          .select(['tailnum','origin','dest']).show(5)

+-------+------+----+
|tailnum|origin|dest|
+-------+------+----+
| N810SK|   SEA| PDX|
| N822SK|   SEA| PDX|
| N586SW|   SEA| PDX|
| N223SW|   SEA| PDX|
| N580SW|   SEA| PDX|
+-------+------+----+
only showing top 5 rows



In [29]:
df_flights.filter((df_flights.origin=='SEA')&(df_flights.dest=='PDX'))\
          .select(['tailnum','origin','dest']).show(5)

+-------+------+----+
|tailnum|origin|dest|
+-------+------+----+
| N810SK|   SEA| PDX|
| N822SK|   SEA| PDX|
| N586SW|   SEA| PDX|
| N223SW|   SEA| PDX|
| N580SW|   SEA| PDX|
+-------+------+----+
only showing top 5 rows



We can perform column-wise operations using `.select()` method. When we select a column using the `df.colName` notation. In `.select()` method, we can perform any column operation and it will return the transformed column. 
For example, the following command returns a column of flight durations in hours instead of minutes.
```python
df_flights.select(df_flights.air_time/60)
```
We can use the `alias()` method to rename a column we've selected. The following example shows how we can do that.
```python
df_flights.select((df_flights.air_time/60).alias("duration_hrs")
```
If we want to stick to the SQL syntax, we can use `.selectExpr()` method as well. The following commad is equivalent to the previous code.

```python
df_flights.selectExpr("air_time/60 as duration_hrs")
```

### Exercise 7: Write a query that return these columns, `origin`, `dest`, `tailnum`, and average speed in KM per hour.

In [30]:
avg_speed =(df_flights.distance/(df_flights.air_time/60)).alias('avg_speed')

In [31]:
df_flights.select('origin','dest','tailnum',avg_speed).show(5)

+------+----+-------+------------------+
|origin|dest|tailnum|         avg_speed|
+------+----+-------+------------------+
|   SEA| LAX| N846VA| 433.6363636363636|
|   SEA| HNL| N559AS| 446.1666666666667|
|   SEA| SFO| N847VA|367.02702702702703|
|   PDX| SJC| N360SW| 411.3253012048193|
|   SEA| BUR| N612AS| 442.6771653543307|
+------+----+-------+------------------+
only showing top 5 rows



In [32]:
df_flights.selectExpr('origin','dest','tailnum','distance/(air_time/60) as avg_speed').show()

+------+----+-------+------------------+
|origin|dest|tailnum|         avg_speed|
+------+----+-------+------------------+
|   SEA| LAX| N846VA| 433.6363636363636|
|   SEA| HNL| N559AS| 446.1666666666667|
|   SEA| SFO| N847VA|367.02702702702703|
|   PDX| SJC| N360SW| 411.3253012048193|
|   SEA| BUR| N612AS| 442.6771653543307|
|   PDX| DEN| N646SW|491.40495867768595|
|   PDX| OAK| N422WN|             362.0|
|   SEA| SFO| N361VA| 415.7142857142857|
|   SEA| SAN| N309AS| 466.6666666666667|
|   SEA| ORD| N564AS| 521.5151515151515|
|   SEA| LAX| N323AS| 440.3076923076923|
|   SEA| PHX| N305AS|431.29870129870125|
|   SEA| LAS| N433AS| 409.6062992125984|
|   SEA| ANC| N765AS|474.75409836065575|
|   SEA| SFO| N713AS| 315.8139534883721|
|   PDX| SFO| N27205| 366.6666666666667|
|   SEA| SMF| N626AS|477.63157894736844|
|   SEA| MDW| N8634A|481.38888888888886|
|   SEA| BOS| N597AS| 516.4137931034483|
|   PDX| BUR| N215AG| 441.6216216216216|
+------+----+-------+------------------+
only showing top

### Exercise 8: Find the the shortest (in terms of distance) flight that left PDX by first filtering and using the `.min()` method. Perform the filtering by referencing the column directly, not passing a SQL string.

In [37]:
df_flights.filter(df_flights.origin=='PDX').groupBy(df_flights.origin)\
          .agg({'distance':'min'}).show()

+------+-------------+
|origin|min(distance)|
+------+-------------+
|   PDX|          106|
+------+-------------+



In [34]:
from pyspark.sql.functions import min
df_flights.filter(df_flights.origin=='PDX').groupBy(df_flights.origin)\
          .agg(min('distance').alias('Shortest Flight Distance')).show()

+------+------------------------+
|origin|Shortest Flight Distance|
+------+------------------------+
|   PDX|                     106|
+------+------------------------+



### Exercise 9: Find the the longest (in terms of time) flight that left SEA by filtering and using the `.max()` method. Perform the filtering by referencing the column directly, not passing a SQL string.

if we run the following code, we will get an error, because `air_time` data type is string, first we should cast it to an integer column.

In [38]:
df_flights.printSchema()


root
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- day: integer (nullable = true)
 |-- dep_time: string (nullable = true)
 |-- dep_delay: string (nullable = true)
 |-- arr_time: string (nullable = true)
 |-- arr_delay: string (nullable = true)
 |-- carrier: string (nullable = true)
 |-- tailnum: string (nullable = true)
 |-- flight: integer (nullable = true)
 |-- origin: string (nullable = true)
 |-- dest: string (nullable = true)
 |-- air_time: integer (nullable = true)
 |-- distance: integer (nullable = true)
 |-- hour: string (nullable = true)
 |-- minute: string (nullable = true)
 |-- duration_hrs: double (nullable = true)



In [39]:
from pyspark.sql.functions import col
df_flights = df_flights.withColumn('air_time',col('air_time').cast('integer'))

In [40]:
from pyspark.sql.functions import max
df_flights.filter(df_flights.origin=='SEA').groupBy(df_flights.origin)\
          .agg(max('air_time').alias('Longest Flight Time')).show()

+------+-------------------+
|origin|Longest Flight Time|
+------+-------------------+
|   SEA|                409|
+------+-------------------+



In [42]:
df_flights.filter(df_flights.origin=='SEA').groupBy(df_flights.origin)\
          .max('air_time').withColumnRenamed('max(air_time)','Longest Flight Time').show()

+------+-------------------+
|origin|Longest Flight Time|
+------+-------------------+
|   SEA|                409|
+------+-------------------+



### Exercise 10: Write a query that uses the `.avg()` method to get the average air time of Delta Airlines flights ( the carrier column value is "DL") that left SEA.

In [43]:
from pyspark.sql.functions import avg
df_flights.filter((df_flights.carrier =='DL') & (df_flights.origin=='SEA')).groupBy(df_flights.carrier)\
          .agg(avg('air_time').alias('Average Flight Time')).show()

+-------+-------------------+
|carrier|Average Flight Time|
+-------+-------------------+
|     DL| 188.20689655172413|
+-------+-------------------+



In [45]:
df_flights.filter(df_flights.origin=='SEA').filter(df_flights.carrier=='DL')\
          .groupBy(df_flights.carrier).avg('air_time').show()

+-------+------------------+
|carrier|     avg(air_time)|
+-------+------------------+
|     DL|188.20689655172413|
+-------+------------------+



In [48]:
def mean_udf(m):
  return round(m.mean(),2)
spark.udf.register('mean_udf',mean_udf)
df_flights.filter((df_flights.carrier =='DL') & (df_flights.origin=='SEA')).groupBy(df_flights.carrier)\
.agg({'air_time':'mean_udf'}).show()

AnalysisException: ignored

### Exercise 11: Write a query that uses the `.sum()` method to get the total number of hours all planes spent in the air by creating a column called `duration_hrs` from the column `air_time`.

In [51]:
hrs= df_flights.air_time/60
new_df=df_flights.withColumn('duration_hrs',hrs).groupBy().sum('duration_hrs')


In [55]:
my_value = new_df.first()[0]

### Exercise 12: Write a query that uses `tailnum` column to count the number of flights each plane made.

In [58]:
df_flights.groupBy('tailnum').count().orderBy('count',ascending=False).show()

+-------+-----+
|tailnum|count|
+-------+-----+
| N612AS|   51|
| N219AG|   50|
| N224AG|   50|
| N218AG|   49|
| N223AG|   49|
| N611AS|   46|
| N615AS|   45|
| N225AG|   45|
| N607AS|   45|
| N435AS|   45|
| N794AS|   42|
| N627AS|   41|
| N215AG|   41|
| N644AS|   41|
| N216AG|   41|
| N227AG|   40|
| N413AS|   40|
| N622AS|   40|
| N626AS|   40|
| N625AS|   40|
+-------+-----+
only showing top 20 rows



### Exercise 13: Write a query that returns the average duration of flights from `PDX` and `SEA`.

In [60]:
df_flights.filter((df_flights.origin=='PDX') | (df_flights.origin=='SEA'))\
          .groupBy(df_flights.origin).agg(round(avg('air_time'),2).alias('Average Flight Time')).show()

+------+-------------------+
|origin|Average Flight Time|
+------+-------------------+
|   SEA|             160.44|
|   PDX|             137.12|
+------+-------------------+



### Exercise 14: Write a query that returns the average departure delay (`dep_delay`) in each month for each destination. Then import PySpark functions to calculate the standard deviation of `dep_delay` by using `stddev()` function.

In [63]:
df_flights.printSchema()

root
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- day: integer (nullable = true)
 |-- dep_time: string (nullable = true)
 |-- dep_delay: integer (nullable = true)
 |-- arr_time: string (nullable = true)
 |-- arr_delay: string (nullable = true)
 |-- carrier: string (nullable = true)
 |-- tailnum: string (nullable = true)
 |-- flight: integer (nullable = true)
 |-- origin: string (nullable = true)
 |-- dest: string (nullable = true)
 |-- air_time: integer (nullable = true)
 |-- distance: integer (nullable = true)
 |-- hour: string (nullable = true)
 |-- minute: string (nullable = true)
 |-- duration_hrs: double (nullable = true)



In [62]:
from pyspark.sql.functions import col
df_flights= df_flights.withColumn('dep_delay',col('dep_delay').cast('integer'))

In [65]:
grouped_by_month_dest = df_flights.groupBy('month','dest')

In [66]:
grouped_by_month_dest.avg('dep_delay').orderBy('month').show(5)

+-----+----+--------------+
|month|dest|avg(dep_delay)|
+-----+----+--------------+
|    1| ABQ|          -3.0|
|    1| SLC|        6.5625|
|    1| MSP|          11.2|
|    1| RDM|        -1.625|
|    1| JNU|           2.6|
+-----+----+--------------+
only showing top 5 rows



In [67]:
from pyspark.sql.functions import stddev
grouped_by_month_dest.agg(stddev('dep_delay')).show(5)

+-----+----+----------------------+
|month|dest|stddev_samp(dep_delay)|
+-----+----+----------------------+
|    4| PHX|    15.003380033491737|
|    1| RDM|     8.830749846821778|
|    5| ONT|    18.895178691342874|
|    7| OMA|    2.1213203435596424|
|    8| MDW|    14.467659032985843|
+-----+----+----------------------+
only showing top 5 rows



### Exercise 15: Write a query that performs left outer join on the flights and airports DataFrames.
- The flights and airports DataFrames are already in the workspace. 
- First, examine the airports DataFrame by calling .show() method. 
- Note which key column will let you join these two DataFrames.
- Before joining these two DataFrames, rename the `faa` column in `airports` to `dest`, and then convert this DataFrame to a temporary view (table).
- Use `spark.sql` to perform left outer join on these two tables. 


In [68]:
df_airports.show(3)

+---+--------------------+----------+-----------+----+---+---+
|faa|                name|       lat|        lon| alt| tz|dst|
+---+--------------------+----------+-----------+----+---+---+
|04G|   Lansdowne Airport|41.1304722|-80.6195833|1044| -5|  A|
|06A|Moton Field Munic...|32.4605722|-85.6800278| 264| -5|  A|
|06C| Schaumburg Regional|41.9893408|-88.1012428| 801| -6|  A|
+---+--------------------+----------+-----------+----+---+---+
only showing top 3 rows



In [69]:
df_flights.show(3)

+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+------------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|duration_hrs|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+------------+
|2014|   12|  8|     658|       -7|     935|       -5|     VX| N846VA|  1780|   SEA| LAX|     132|     954|   6|    58|         2.2|
|2014|    1| 22|    1040|        5|    1505|        5|     AS| N559AS|   851|   SEA| HNL|     360|    2677|  10|    40|         6.0|
|2014|    3|  9|    1443|       -2|    1652|        2|     VX| N847VA|   755|   SEA| SFO|     111|     679|  14|    43|        1.85|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+------------+
only showing top 3 rows



In [70]:
df_airports=df_airports.withColumnRenamed('faa','dest')
df_airports.show(3)

+----+--------------------+----------+-----------+----+---+---+
|dest|                name|       lat|        lon| alt| tz|dst|
+----+--------------------+----------+-----------+----+---+---+
| 04G|   Lansdowne Airport|41.1304722|-80.6195833|1044| -5|  A|
| 06A|Moton Field Munic...|32.4605722|-85.6800278| 264| -5|  A|
| 06C| Schaumburg Regional|41.9893408|-88.1012428| 801| -6|  A|
+----+--------------------+----------+-----------+----+---+---+
only showing top 3 rows



In [71]:
df_flights.createOrReplaceTempView('flights')
df_airports.createOrReplaceTempView('airports')

In [72]:
df_joined =spark.sql("SELECT fl.origin,fl.dest, ap.name, fl.flight FROM\
                      flights as fl LEFT JOIN airports as ap ON fl.dest=ap.dest")
df_joined.show()

+------+----+--------------------+------+
|origin|dest|                name|flight|
+------+----+--------------------+------+
|   SEA| LAX|    Los Angeles Intl|  1780|
|   SEA| HNL|       Honolulu Intl|   851|
|   SEA| SFO|  San Francisco Intl|   755|
|   PDX| SJC|Norman Y Mineta S...|   344|
|   SEA| BUR|            Bob Hope|   522|
|   PDX| DEN|         Denver Intl|    48|
|   PDX| OAK|Metropolitan Oakl...|  1520|
|   SEA| SFO|  San Francisco Intl|   755|
|   SEA| SAN|      San Diego Intl|   490|
|   SEA| ORD|  Chicago Ohare Intl|    26|
|   SEA| LAX|    Los Angeles Intl|   448|
|   SEA| PHX|Phoenix Sky Harbo...|   656|
|   SEA| LAS|      Mc Carran Intl|   608|
|   SEA| ANC|Ted Stevens Ancho...|   121|
|   SEA| SFO|  San Francisco Intl|   306|
|   PDX| SFO|  San Francisco Intl|  1458|
|   SEA| SMF|     Sacramento Intl|   368|
|   SEA| MDW| Chicago Midway Intl|   827|
|   SEA| BOS|General Edward La...|    24|
|   PDX| BUR|            Bob Hope|  3488|
+------+----+--------------------+

### Exercise 16: Rewrite the previous query by using DataFrame API `.join` method. 
In PySpark, we can use `.join` method to perform joins. This method takes three arguments. 
- The first argument is the second DataFrame that we want to join with the first one. 
- The second argument, `on`, is the name of the key column(s) as a string. The names of the key column(s) must be the same in each table. 
- The third argument, `how`, specifies the kind of join to perform. 

To perform left outer join set the value of `how` to `"leftouter"`.

In [74]:
df_flights.join(df_airports,'dest','leftouter').select('origin', 'dest','name','flight').show()

+------+----+--------------------+------+
|origin|dest|                name|flight|
+------+----+--------------------+------+
|   SEA| LAX|    Los Angeles Intl|  1780|
|   SEA| HNL|       Honolulu Intl|   851|
|   SEA| SFO|  San Francisco Intl|   755|
|   PDX| SJC|Norman Y Mineta S...|   344|
|   SEA| BUR|            Bob Hope|   522|
|   PDX| DEN|         Denver Intl|    48|
|   PDX| OAK|Metropolitan Oakl...|  1520|
|   SEA| SFO|  San Francisco Intl|   755|
|   SEA| SAN|      San Diego Intl|   490|
|   SEA| ORD|  Chicago Ohare Intl|    26|
|   SEA| LAX|    Los Angeles Intl|   448|
|   SEA| PHX|Phoenix Sky Harbo...|   656|
|   SEA| LAS|      Mc Carran Intl|   608|
|   SEA| ANC|Ted Stevens Ancho...|   121|
|   SEA| SFO|  San Francisco Intl|   306|
|   PDX| SFO|  San Francisco Intl|  1458|
|   SEA| SMF|     Sacramento Intl|   368|
|   SEA| MDW| Chicago Midway Intl|   827|
|   SEA| BOS|General Edward La...|    24|
|   PDX| BUR|            Bob Hope|  3488|
+------+----+--------------------+

## Well Done!